# 首先创建一个chat llm without rag

In [14]:
import os
from langchain.chat_models import ChatOpenAI

chatLLM = ChatOpenAI(
    api_key = "sk-1c33a0011b5c4ef6ac842d58255a20d8",
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    model = "qwen2-7b-instruct",  # 此处为了演示幻觉，以qwen2-7b-instruct为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你是谁？"}]
response = chatLLM.invoke(messages)
print(response)
print(response.content)

content='我是阿里云开发的一款超大规模语言模型，我叫通义千问。作为一个AI助手，我的目标是帮助用户获得准确、有用的信息，解决他们的问题和困惑。无论是关于科技、教育、生活常识还是其他领域的问题，我都会尽力提供最合适的帮助。请随时向我提问，我会努力回答。' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 22, 'total_tokens': 91, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2-7b-instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--9b0cdda3-2c0d-44cb-9d31-9c260f34f86f-0'
我是阿里云开发的一款超大规模语言模型，我叫通义千问。作为一个AI助手，我的目标是帮助用户获得准确、有用的信息，解决他们的问题和困惑。无论是关于科技、教育、生活常识还是其他领域的问题，我都会尽力提供最合适的帮助。请随时向我提问，我会努力回答。


# 处理LLM存在的缺陷问题

- 容易出现幻觉
- 数据信息时效性滞后
- 专业领域深度知识匮乏

In [16]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你知道baichuan2模型吗？"}
]

In [17]:
response = chatLLM.invoke(messages)
print(response.content)

是的，我知道Baichuan 2。Baichuan 2是由阿里云开发的一个大型语言模型，全名是“通义千问”。它是基于阿里云的超大规模神经网络训练而成，旨在通过学习大量的文本数据来生成与之类似的文字内容。相较于其前身，Baichuan 2在性能、准确性和生成质量上都有了显著提升。

作为阿里云的AI助手，通义千问可以回答各种问题、提供信息、进行对话，并且能够根据不同的场景和需求生成相应的文本内容。例如，它可以用于编写故事、解答技术问题、提供商业策略建议等。它具备一定的语言理解能力，能够对用户的输入进行分析，并基于此提供相关回复或执行相应的任务。


In [18]:
baichuan2_information = [
    "Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。",
    "Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。",
    "Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。"
]

source_knowledge = "\n".join(baichuan2_information)

In [19]:
source_knowledge

'Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。\nBaichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。\nBaichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。'

In [21]:
query = "你知道baichuan2模型吗？"

prompt_template = f"""基于以下内容回答问题：

内容:
{source_knowledge}

Query: {query}"""

In [22]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt_template}
]

In [23]:
response = chatLLM.invoke(messages)
print(response.content)

Baichuan 2是一个大型多语言语言模型，专注于训练在多种语言中表现优异的模型，包括但不限于英文。它从头开始训练，使用了包含2.6万亿个标记的大量训练数据集，相比以往的模型提供了更丰富的数据资源，从而更好地支持多语言的开发和应用。Baichuan 2在通用任务和特定领域（如医学和法律）的任务中都表现出色，为特定领域的应用提供了有力支持。


In [24]:
# 加载pdf数据
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305v2.pdf")
# loader = PyPDFLoader("./pdf/2309.10305v2.pdf")
pages = loader.load_and_split()

In [25]:
print(len(pages))
pages[0]

39


Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-09-21T00:15:31+00:00', 'author': '', 'keywords': '', 'moddate': '2023-09-21T00:15:31+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'https://arxiv.org/pdf/2309.10305v2.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}, page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun\nTao Zhang, Tianpeng Li, Tianyu Li, Wei Cheng, Weipeng Chen, Xiangrong Zeng\nXiaochuan

In [26]:
# 知识切片 将文档分割成均匀的块。每个块是一段原始文本
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)
len(docs)

216

In [35]:
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.vectorstores import Chroma

embed_model = DashScopeEmbeddings(
    model = "text-embedding-v2",
    dashscope_api_key = "sk-1c33a0011b5c4ef6ac842d58255a20d8"
)
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embed_model,
    collection_name="brench_embed_dev",
    persist_directory="./chroma_data_baichuan2_rag_demo"
)

In [51]:
query = "百川2大模型是什么，以及百川2大模型用的参数大小?"
result = vectorstore.similarity_search(query ,k = 2)
result

[Document(metadata={'page_label': '2', 'subject': '', 'creationdate': '2023-09-21T00:15:31+00:00', 'creator': 'LaTeX with hyperref', 'moddate': '2023-09-21T00:15:31+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'total_pages': 28, 'author': '', 'source': 'https://arxiv.org/pdf/2309.10305v2.pdf', 'producer': 'pdfTeX-1.40.25', 'trapped': '/False', 'title': '', 'keywords': '', 'page': 1}, page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,'),
 Document(metadata={'creationdate': '2023-09-21T00:15:31+00:0

In [52]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: {query}"""
  return augmented_prompt

In [53]:
print(augment_prompt(query))

Using the contexts below, answer the query.

  contexts:
  languages, such as Chinese.
In this technical report, we introduce Baichuan
2, a series of large-scale multilingual language
models. Baichuan 2 has two separate models,
Baichuan 2-7B with 7 billion parameters and
Baichuan 2-13B with 13 billion parameters. Both
models were trained on 2.6 trillion tokens, which
to our knowledge is the largest to date, more than
double that of Baichuan 1 (Baichuan, 2023b,a).
With such a massive amount of training data,
overall performance of the Baichuan 2 base models
compared to other open or closed-sourced models
A Scaling laws
We use 7 models to fit the scaling laws of Baichuan
2. The parameter details are shown in Table 10.
Nhidden NFFN Nlayer Nhead Nparams (Millions)
384 1,152 6 6 11.51
704 2,112 8 8 51.56
832 2,496 12 8 108.01
1,216 3,648 16 8 307.60
1,792 5,376 20 14 835.00
2,240 6,720 24 14 1,565.60
2,880 8,640 28 20 3,019.33
Table 10: The model we choose for fitting scaling laws.
The loss

In [54]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": augment_prompt(query)}
]

In [55]:
response = chatLLM.invoke(messages)
print(response.content)

百川2是多语言大规模语言模型系列的一种。它包含了两个独立的模型：百川2-7B，拥有70亿个参数；和百川2-13B，拥有130亿个参数。这两个模型都是在26万亿个标记上进行训练的，据我们所知，这是迄今为止最大的数据集，超过Baichuan 1的两倍（Baichuan, 2023b,a）。通过如此大量的训练数据，百川2基模的整体性能与其他开源或闭源模型相比有了显著提升。

在研究中使用了7个模型来拟合百川2的扩展法则。这些模型的详细参数如表10所示：

| 隐藏层大小 | FFN（前馈神经网络）数量 | 层数 | 头数 | 参数（百万） |
| --- | --- | --- | --- | --- |
| 384 | 1,152 | 6 | 6 | 11.51 |
| 704 | 2,112 | 8 | 8 | 51.56 |
| 832 | 2,496 | 12 | 8 | 108.01 |
| 1,216 | 3,648 | 16 | 8 | 307.60 |
| 1,792 | 5,376 | 20 | 14 | 835.00 |
| 2,240 | 6,720 | 24 | 14 | 1,565.60 |
| 2,880 | 8,640 | 28 | 20 | 3,019.33 |

表10展示了用于拟合扩展法则的模型选择。不同模型的损失值在图8中有所显示。

因此，百川2大模型是指百川2-7B（70亿参数）和百川2-13B（130亿参数）两种模型。
